In [1]:
import pandas as pd

# Step 1: Define path
excel_path = '/Users/emilsafarov/Library/CloudStorage/OneDrive-Personal/CF/A6/1_sourceing_open_data/US project/Data/Output data/untitled folder/essential.xlsx'

# Step 2: Load all sheets into a dictionary
all_sheets = pd.read_excel(excel_path, sheet_name=None)


In [2]:
len(all_sheets)


49

In [3]:
for sheet_name, df in all_sheets.items():
    print(f"Sheet: {sheet_name} → {len(df)} rows")


Sheet: Population → 3144 rows
Sheet: CO-EST2024-POP → 3144 rows
Sheet: air_pollution → 3198 rows
Sheet: children_lunch → 3198 rows
Sheet: children_poverty → 3198 rows
Sheet: dentists_raw_value → 3198 rows
Sheet: disconnected_youth → 3198 rows
Sheet: drinking_water → 3199 rows
Sheet: drive_alone → 3198 rows
Sheet: drug_overdose_deaths → 3198 rows
Sheet: exercise_access → 3198 rows
Sheet: firearm_fatalities → 3198 rows
Sheet: flu_vaccinations_raw_value → 3198 rows
Sheet: food_environment_index → 3198 rows
Sheet: food_insecurity → 3198 rows
Sheet: frequent_physical_distress → 3198 rows
Sheet: high_housing_cost → 3198 rows
Sheet: high_school_graduation → 3198 rows
Sheet: hiv_prevalence_raw_value → 3198 rows
Sheet: homeownership → 3198 rows
Sheet: homicides → 3198 rows
Sheet: housing_facilities → 3198 rows
Sheet: income_inequality → 3198 rows
Sheet: injury_deaths → 3198 rows
Sheet: insufficient_sleep_raw_value → 3198 rows
Sheet: juvenile_arrests → 3198 rows
Sheet: life_expectancy → 3198 row

In [4]:
# 🔁 Loop through target sheets and check for duplicate FIPS codes
for sheet in target_sheets_1:
    df = all_sheets[sheet].copy()
    
    if 'FIPS' in df.columns:
        # Standardize FIPS to 5-digit strings
        df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)
        
        # Count duplicates
        duplicates = df['FIPS'].duplicated().sum()
        
        # Report if any found
        if duplicates > 0:
            print(f"❗ Sheet '{sheet}' has {duplicates} duplicate FIPS")
    else:
        print(f"⚠️ Sheet '{sheet}' has no 'FIPS' column")


NameError: name 'target_sheets_1' is not defined

In [ ]:
target_sheets_1 = [
    "air_pollution", "children_lunch", "children_poverty", "dentists_raw_value",
    "disconnected_youth", "drinking_water", "drive_alone", "drug_overdose_deaths",
    "exercise_access", "firearm_fatalities", "flu_vaccinations_raw_value", "food_environment_index",
    "food_insecurity", "frequent_physical_distress", "high_housing_cost", "high_school_graduation",
    "hiv_prevalence_raw_value", "homeownership", "homicides", "housing_facilities",
    "income_inequality", "injury_deaths", "insufficient_sleep_raw_value", "juvenile_arrests",
    "life_expectancy", "limited_access_healthy_foods", "long_commute", "mammography_screening",
    "math_scores", "median_income", "mental_health_providers_raw_val", "motor_vehicle_crash_deaths",
    "not_proficient_english", "poor_or_fair_health", "poor_physical_health", "population_raw_value",
    "primary_care_physicians", "reading_scores", "severe_housing", "severe_housing_problems",
    "some_college", "suicides_raw_value", "traffic_volume", "unemployment",
    "uninsured_raw_value", "violent_crime"
]


In [ ]:
# Initialize merged DataFrame
merged_df_1 = None

# Merge all sheets side-by-side on FIPS
for sheet_name in target_sheets_1:
    df = all_sheets[sheet_name].copy()

    # Standardize FIPS
    df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)

    # Keep only one row per FIPS, State, County (take mean of numeric values)
    group_keys = ['FIPS']
    if 'State' in df.columns and 'County' in df.columns:
        group_keys = ['FIPS', 'State', 'County']

    df = df.groupby(group_keys, as_index=False).mean(numeric_only=True)

    # Rename value columns (but keep FIPS, State, County intact)
    df = df.rename(columns={col: f"{sheet_name}__{col}" for col in df.columns if col not in group_keys})

    # Merge
    if merged_df_1 is None:
        merged_df_1 = df
    else:
        merged_df_1 = pd.merge(merged_df_1, df, on=group_keys, how="outer")

# ✅ Final merged_df_1 contains FIPS, State, County, and all indicators
print("✅ Final merged shape:", merged_df_1.shape)
print(merged_df_1.head())


In [ ]:
merged_df_1

In [ ]:
# Load the two population-related sheets
pop_df_1 = all_sheets["Population"].copy()
pop_df_2 = all_sheets["CO-EST2024-POP"].copy()

# Standardize column names
pop_df_1.columns = pop_df_1.columns.str.strip()
pop_df_2.columns = pop_df_2.columns.str.strip()

# Merge on State and County
merged_df_2 = pd.merge(pop_df_1, pop_df_2, on=["State", "County"], how="outer", suffixes=("_pop", "_est2024"))

# ✅ Preview result
print("✅ Merged shape:", merged_df_2.shape)
print(merged_df_2.head())


In [ ]:
merged_df_2

In [ ]:
# Extract unique FIPS–State–County mapping
fips_lookup_df = merged_df_1[['FIPS', 'State', 'County']].drop_duplicates()


In [ ]:
# Add FIPS to merged_df_2 using State + County as the key
merged_df_2 = pd.merge(merged_df_2, fips_lookup_df, on=["State", "County"], how="left")

# Confirm FIPS added
print("✅ FIPS added to merged_df_2")
print(merged_df_2[['FIPS', 'State', 'County']].head())


In [ ]:
# 🔍 Filter rows where FIPS was not found (NaN after merge)
missing_fips_rows = merged_df_2[merged_df_2['FIPS'].isna()]

# 📊 Show how many are missing
print(f"❗ Rows with missing FIPS: {missing_fips_rows.shape[0]}")

# 👀 Preview them
print(missing_fips_rows[['State', 'County']])


In [ ]:
fmr_df = all_sheets["fmr"].copy()

# Make sure FIPS is standardized
fmr_df['FIPS'] = fmr_df['FIPS'].astype(str).str.zfill(5)

# (Optional) Check available columns
# print(fmr_df.columns)


In [ ]:
# Extract State + County from missing rows
missing_fips_rows = merged_df_2[merged_df_2['FIPS'].isna()][['State', 'County']].drop_duplicates()

# Check if these rows exist in merged_df_1
merged_1_check = pd.merge(missing_fips_rows, merged_df_1[['FIPS', 'State', 'County']], on=['State', 'County'], how='left')
print("🔎 FIPS from merged_df_1:")
print(merged_1_check)

# Check if they exist in fmr_df
fmr_check = pd.merge(missing_fips_rows, fmr_df[['FIPS', 'State', 'County']], on=['State', 'County'], how='left')
print("\n🔎 FIPS from fmr_df:")
print(fmr_check)


In [ ]:
# Dictionary of manually fixed FIPS codes
manual_fips_fixes = {
    ('AK', 'Anchorage Municipality'): '02020',
    ('AK', 'Juneau City and Borough'): '02110',
    ('AK', 'Petersburg Borough'): '02195',
    ('AK', 'Sitka City and Borough'): '02220',
    ('AK', 'Wrangell City and Borough'): '02275',
    ('AK', 'Yakutat City and Borough'): '02282',
    
    ('CT', 'Capitol Planning Region'): '00001',
    ('CT', 'Greater Bridgeport Planning Region'): '00002',
    ('CT', 'Lower Connecticut River Valley Planning Region'): '00003',
    ('CT', 'Naugatuck Valley Planning Region'): '00004',
    ('CT', 'Northeastern Connecticut Planning Region'): '00005',
    ('CT', 'Northwest Hills Planning Region'): '00006',
    ('CT', 'South Central Connecticut Planning Region'): '00007',
    ('CT', 'Southeastern Connecticut Planning Region'): '00008',
    ('CT', 'Western Connecticut Planning Region'): '00009',

    ('IL', 'LaSalle County'): '17099',
    ('IN', 'DeKalb County'): '18033',
    ('IN', 'LaGrange County'): '18087',
    ('IN', 'LaPorte County'): '18091',
    ('NM', 'De Baca County'): '35011',
    ('NM', 'Doña Ana County'): '35013',
    ('PA', 'McKean County'): '42083'
}

# Apply manual corrections
merged_df_2['FIPS'] = merged_df_2.apply(
    lambda row: manual_fips_fixes.get((row['State'], row['County']), row['FIPS']),
    axis=1
)


In [ ]:
merged_df_2

In [ ]:
# 🔍 Filter rows where FIPS, State, or County is missing
missing_key_info = merged_df_2[
    merged_df_2[['FIPS', 'State', 'County']].isna().any(axis=1)
]

# 👀 Display them
print("❗ Rows with missing FIPS, State, or County:")
print(missing_key_info[['FIPS', 'State', 'County']])


In [ ]:
# 🔍 Count how many FIPS in merged_df_1 also exist in merged_df_2
# This tells you how many rows will successfully match during the merge
print("🔍 FIPS match in merged_df_2:", merged_df_1['FIPS'].isin(merged_df_2['FIPS']).sum())

# 🔍 Count how many FIPS in merged_df_1 also exist in fmr_df
# Useful to ensure FMR data can be merged without losing rows
print("🔍 FIPS match in fmr_df:", merged_df_1['FIPS'].isin(fmr_df['FIPS']).sum())


In [ ]:
print("📦 merged_df_1 (Main dataset) rows:", merged_df_1.shape[0])
print("📦 merged_df_2 (Population data) rows:", merged_df_2.shape[0])
print("📦 fmr_df (Fair Market Rent) rows:", fmr_df.shape[0])


In [ ]:
# 🔍 Find duplicated rows based on FIPS, State, and County
fmr_duplicates = fmr_df[fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)]

# 📊 Show how many duplicated entries
print(f"❗ Duplicated rows in fmr_df: {fmr_duplicates.shape[0]}")

# 👀 Display the duplicated rows
print(fmr_duplicates.sort_values(by=['FIPS', 'State', 'County']))


In [ ]:
# 🔍 Find rows with duplicated FIPS only
fmr_dup_fips = fmr_df[fmr_df.duplicated(subset='FIPS', keep=False)]

# 📊 Show how many FIPS codes are duplicated
print(f"❗ Rows with duplicated FIPS in fmr_df: {fmr_dup_fips.shape[0]}")

# 👀 Display the duplicated rows sorted by FIPS
print(fmr_dup_fips.sort_values(by='FIPS'))


In [ ]:
import pandas as pd

# Step 1: Create FIPS lookup from merged_df_1 and rename FIPS
fips_lookup_df = merged_df_1[['State', 'County', 'FIPS']].drop_duplicates()
fips_lookup_df = fips_lookup_df.rename(columns={'FIPS': 'FIPS_corrected'})

# Step 2: Merge into fmr_df using State + County
fmr_df = pd.merge(
    fmr_df,
    fips_lookup_df,
    on=['State', 'County'],
    how='left'  # Keep all rows from fmr_df
)

# Step 3: Show unmatched rows (where merge failed = no FIPS_corrected found)
mismatched_names = fmr_df[fmr_df['FIPS_corrected'].isna()]

print(f"❗ Rows with unmatched State + County: {mismatched_names.shape[0]}")
print(mismatched_names[['State', 'County', 'FIPS']])

# Optional: Show rows where FIPS exists but differs from corrected one
fips_conflicts = fmr_df[
    (fmr_df['FIPS_corrected'].notna()) &
    (fmr_df['FIPS'] != fmr_df['FIPS_corrected'])
]

print(f"\n🔁 Rows with differing FIPS values: {fips_conflicts.shape[0]}")
print(fips_conflicts[['State', 'County', 'FIPS', 'FIPS_corrected']])


In [ ]:
fmr_df

In [ ]:
# Step 1: Manually confirmed FIPS values
manual_fips_fixes_2 = {
    ('IN', 'DeKalb County'): '18033',
    ('IN', 'LaGrange County'): '18087',
    ('IN', 'LaPorte County'): '18091',
    ('AK', 'Anchorage Municipality'): '02020',
    ('AK', 'Juneau City and Borough'): '02110',
    ('AK', 'Petersburg Borough'): '02195',
    ('AK', 'Sitka City and Borough'): '02220',
    ('AK', 'Wrangell City and Borough'): '02275',
    ('AK', 'Yakutat City and Borough'): '02282',
    ('MO', 'Sullivan part'): '29211',  # Closest FIPS: Sullivan County, MO
    ('NM', 'De Baca County'): '35011',
    ('PA', 'McKean County'): '42083'
}


In [ ]:
# Step 2: Update FIPS_corrected using manual fixes only where FIPS_corrected is missing
fmr_df['FIPS_corrected'] = fmr_df.apply(
    lambda row: manual_fips_fixes.get((row['State'], row['County']), row['FIPS_corrected']),
    axis=1
)


In [ ]:
fmr_df

In [ ]:
# Show all columns
print(fmr_df.columns.tolist())


In [ ]:
# Step 1: Drop all other FIPS_corrected columns except the last one
fmr_df = fmr_df.drop(columns=[col for col in fmr_df.columns if col.startswith('FIPS_corrected_')])

# Step 2: Rename the surviving one to keep it clear
fmr_df = fmr_df.rename(columns={'FIPS_corrected': 'FIPS_corrected_clean'})

# Step 3 (optional): Create final FIPS column — fallback to original if corrected is missing
fmr_df['FIPS_final'] = fmr_df['FIPS_corrected_clean'].combine_first(fmr_df['FIPS'])


In [ ]:
fmr_df

In [ ]:
# Drop both FIPS and corrected version (after FIPS_final has been created)
fmr_df = fmr_df.drop(columns=['FIPS', 'FIPS_corrected_clean'])

# ✅ Check result
print(fmr_df.columns.tolist())


In [ ]:
# Step 1: Drop original FIPS columns
fmr_df = fmr_df.drop(columns=['FIPS', 'FIPS_corrected_clean'], errors='ignore')

# Step 2: Rename final column
fmr_df = fmr_df.rename(columns={'FIPS_final': 'FIPS'})

# Step 3: Reorder — move FIPS to 3rd column
cols = fmr_df.columns.tolist()
fips_index = cols.index('FIPS')
# Remove FIPS and insert at index 2 (3rd position)
cols.insert(2, cols.pop(fips_index))
fmr_df = fmr_df[cols]

# ✅ Preview
print(fmr_df.columns[:5])


In [ ]:
fmr_df

In [ ]:
# Check full row duplicates
full_duplicates = fmr_df.duplicated()
print(f"❗ Full duplicate rows: {full_duplicates.sum()}")

# Check duplicates based on FIPS, State, and County
key_duplicates = fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)
print(f"❗ Duplicate rows based on FIPS + State + County: {key_duplicates.sum()}")

# Optional: Show those rows
if key_duplicates.sum() > 0:
    print(fmr_df[key_duplicates].sort_values(by=['FIPS', 'State', 'County']))


In [ ]:
# Step 1: Count full duplicates
full_dups = fmr_df.duplicated()
print(f"❗ Fully duplicated rows to drop: {full_dups.sum()}")

# Step 2: Drop them
fmr_df = fmr_df.drop_duplicates()

# ✅ Confirm new shape
print(f"✅ fmr_df shape after dropping full duplicates: {fmr_df.shape}")


In [ ]:
# Check for duplicate rows based on FIPS, State, and County
key_duplicates = fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)

# Filter and show
duplicate_rows = fmr_df[key_duplicates].sort_values(by=['FIPS', 'State', 'County'])
print(duplicate_rows)


In [ ]:
# Step 1: Group by FIPS, State, County and average all other columns
fmr_df = fmr_df.groupby(['FIPS', 'State', 'County'], as_index=False).mean(numeric_only=True)

# ✅ Confirm shape after deduplication
print(f"✅ Shape after resolving duplicates: {fmr_df.shape}")


In [ ]:
# 🔍 Check for fully duplicated rows
full_dups = fmr_df.duplicated()
print(f"❗ Full duplicate rows: {full_dups.sum()}")

# 🔍 Check for duplicates based on FIPS, State, County
key_dups = fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)
print(f"❗ Duplicate rows based on FIPS + State + County: {key_dups.sum()}")


In [ ]:
print(f"merged_df_1 rows: {merged_df_1.shape[0]}")
print(f"merged_df_2 rows: {merged_df_2.shape[0]}")
print(f"fmr_df rows: {fmr_df.shape[0]}")


In [ ]:
# 🧱 Step 1: Start with merged_df_1 as the base
# We'll merge everything into this DataFrame

# 📦 Step 2: Merge population data (merged_df_2) using FIPS, State, County
df_merged = pd.merge(
    merged_df_1,
    merged_df_2,
    on=['FIPS', 'State', 'County'],
    how='left',  # Keep all rows from merged_df_1, even if there's no match
    suffixes=('', '_pop')
)

# 🏠 Step 3: Merge FMR data (fmr_df) using FIPS, State, County
df_merged = pd.merge(
    df_merged,
    fmr_df,
    on=['FIPS', 'State', 'County'],
    how='left',  # Again, keep all rows from base dataset
    suffixes=('', '_fmr')
)

# ✅ Step 4: Confirm shape and preview
print(f"✅ Final merged dataset shape: {df_merged.shape}")
print(df_merged[['FIPS', 'State', 'County']].head())


In [ ]:
# Step 1: Check missing values
missing_summary = df_merged.isna().sum()
missing_summary = missing_summary[missing_summary > 0]

# Step 2: Identify unmatched rows from merged_df_2 and fmr_df
merged_2_columns = [col for col in merged_df_2.columns if col not in ['FIPS', 'State', 'County']]
fmr_columns = [col for col in fmr_df.columns if col not in ['FIPS', 'State', 'County']]

missing_merged_2 = df_merged[merged_2_columns].isna().all(axis=1).sum()
missing_fmr = df_merged[fmr_columns].isna().all(axis=1).sum()

# Step 3: Check for key-based duplicates
dup_count = df_merged.duplicated(subset=['FIPS', 'State', 'County']).sum()

# Step 4: Print summary
summary = {
    "Total rows": df_merged.shape[0],
    "Rows missing any value": int(df_merged.isna().any(axis=1).sum()),
    "Rows missing all merged_df_2 columns": int(missing_merged_2),
    "Rows missing all fmr_df columns": int(missing_fmr),
    "Duplicate rows (FIPS + State + County)": int(dup_count),
    "Columns with missing values": missing_summary.to_dict()
}
print(summary)


In [ ]:
missing_rows = df_merged[df_merged.isna().any(axis=1)]
missing_rows.to_excel("missing_values_review.xlsx", index=False)


In [ ]:
df_merged['missing_fmr'] = df_merged[fmr_columns].isna().all(axis=1)
df_merged['missing_demo'] = df_merged[merged_2_columns].isna().all(axis=1)


In [ ]:
# Set your export folder path
export_folder = "/Users/emilsafarov/Library/CloudStorage/OneDrive-Personal/CF/A6/1_sourceing_open_data/US project/Data/Output data/untitled folder"

# Export as Excel (.xlsx)
df_merged.to_excel(f"{export_folder}/final_merged_dataset.xlsx", index=False)

# Export as CSV
df_merged.to_csv(f"{export_folder}/final_merged_dataset.csv", index=False)

print("✅ Exported both Excel and CSV to the output folder.")
